# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 10
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus Polling - FIFO
- der Algorithmus weist den BEVs jeweils ein Zustand zu, die Zustände sind wartend (BEV parkt und will laden), ladend (BEV wird mit Solarenergie geladen), nicht parkend (BEV befindet sich nicht im Lademanagmentsystem)
- pro BEV Ladeleistung von z.B. 2kW (ladeleistung_pro_bev), wenn verfügbarer Solarstrom > Ladeleistung, dann wird der Solarstrom auf mehrere BEVs verteilt
- aktuell wartende Verbraucher werden in einer Warteliste (Pollingliste) abgearbeitet, nach einem variablen Zeitintervall (maximale_ladezeit, z.B. 30min) wird Verbraucher wieder hinten an die Warteliste angefügt. 
- dabei wird immer das erste BEV der Warteliste auf die Ladeliste genommen, unabhängig von der Länge der Parkzeit
- Wenn im Tagesverlauf die Anzahl bereits ladender BEVs größer wird als die Anzahl möglich ladender BEVs, wird das erste BEV der Ladeliste wieder nach hinten an die Warteliste gesetzt

### Verteilungsalgorithmus Polling - mit gleichmäßiger Verteilung
- wie FIFO Abarbeitung der BEVs auf der Warteliste
- allerdings werden BEVs nach dem sie einmal aufgeladen wurden auf eine neue Warteliste gesetzt mit geringerer Priorität => so soll gewährleistet werden, dass erstmal alle BEVs einmal aufgeladen werden, bevor BEVs mehrmals geladen werden 

### Prognose (Berücksichtigung der Parkdauer - Optimale Ausnutzung)
- Ziel ist das möglichst alle gleich viel bekommen und der Solarstrom optimal ausgenutzt wird ⇒ nichts geht verloren
- Berechnung des Ladeintervalls:
    - Wann sind während des angegebenen Parkintervalls am wenigsten andere BEVs am Warten? (Minimum Anzahl wartender BEVs in Parkintevall)
    - Ladestart: sobald Minimum Anzahl wartender BEVs in Parkintevall erreicht wird
    - Ladeende: nach fester Zeit oder nachdem Anzahl wartender BEVs nicht mehr Minimum Anzahl wartender BEVs entspricht
- Solarleistung wird zu jedem Zeitpunkt gleichmäßig unter ladenden BEVs verteilt, also Ladeleistung = Verfügbare Solarleistung / Ladende BEVs

### Prognose (Berücksichtigung der Parkdauer - Belohnung)
- Ziel: länger parkende BEVs sollen belohnt werden und mehr Solarenergie garantiert bekommen
- Möglichkeiten: mehr Solarenergie oder längeres Ladeintervall

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm)

distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
                    bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

Start: BEVs_dict {0: [(12.5, 7.3), 'wartend', [], []], 1: [(8.0, 5.6), 'nicht parkend', [], []], 2: [(8.0, 3.8), 'nicht parkend', [], []], 3: [(12.5, 3.5), 'nicht parkend', [], []], 4: [(8.0, 7.4), 'nicht parkend', [], []], 5: [(13.0, 6.1), 'wartend', [], []], 6: [(13.0, 4.1), 'wartend', [], []], 7: [(12.5, 4.1), 'wartend', [], []], 8: [(13.0, 6.6), 'wartend', [], []], 9: [(8.5, 7.1), 'nicht parkend', [], []]}
Updated BEVs_dict {0: [(12.5, 7.3), 'wartend', [], []], 1: [(8.0, 5.6), 'nicht parkend', [], []], 2: [(8.0, 3.8), 'nicht parkend', [], []], 3: [(12.5, 3.5), 'nicht parkend', [], []], 4: [(8.0, 7.4), 'nicht parkend', [], []], 5: [(13.0, 6.1), 'wartend', [], []], 6: [(13.0, 4.1), 'wartend', [], []], 7: [(12.5, 4.1), 'wartend', [], []], 8: [(13.0, 6.6), 'wartend', [], []], 9: [(8.5, 7.1), 'nicht parkend', [], []]}


für BEV mit ID 0
0.9571666666666668 Fairer Anteil für BEV
{750: [5, 0.15056666666666668], 765: [4, 0.16266666666666665], 780: [4, 0.15], 795: [4, 0.15020833333333333], 8

{780: [0, 0.15], 795: [0, 0.15020833333333333], 810: [2, 0.16325], 825: [3, 0.17595833333333333], 840: [3, 0.175], 855: [3, 0.16041666666666665], 870: [2, 0.17627777777777776], 885: [3, 0.1338888888888889], 900: [2, 0.15], 915: [2, 0.11199999999999999], 930: [1, 0.17366666666666666], 945: [1, 0.149], 960: [1, 0.15], 975: [1, 0.17666666666666667], 990: [2, 0.11441666666666667], 1005: [2, 0.15341666666666667], 1020: [3, 0.13677777777777778], 1035: [3, 0.17983333333333332], 1050: [4, 0.1735833333333333], 1065: [5, 0.1749666666666667], 1080: [6, 0.18016666666666667], 1095: [8, 0.16410416666666666], 1110: [9, 0.17446296296296293], 1125: [11, 0.1684848484848485], 1140: [12, 0.18016666666666667]} Forecast-dict
[[], [], [810, 825, 840, 855, 870, 885, 900, 915, 930, 945, 960, 975, 990, 1005, 1020, 1035, 1050, 1065, 1080, 1095, 1110, 1125, 1140]] Possible charging intervals list
{0: 0, 1: 0, 2: 3.6965047558922555} Charging Energies dict
2 index_max_charging_energy
Updated BEVs_dict {0: [(12.5, 7

AttributeError: 'list' object has no attribute 'set_charging_tuple'

### Ergebnis Algorithmus

In [ ]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [ ]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

In [ ]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day)

In [ ]:
figureGeneration.create_bev_number_figure(simulation_data)